In [ ]:
#### Without Pre Process

In [2]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
from keras.applications.xception import preprocess_input

from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

from keras.applications.xception import Xception

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_Xception_data4.h5'

train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Xception network
    model = Xception(weights='imagenet', include_top=False)
    
    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_Xception_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_Xception_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_Xception_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_Xception_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_train shape',y_pred_validation.shape)
    print(' y_pred_train : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

Epoch 39/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 40/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 41/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 42/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 43/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 44/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 45/50
1000/1000 [==============================] - 28s 28ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 46/50
1000/1000 [==================

/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


validation loss: 7.971192412376422
validation accuracy: 0.5
 y_pred_train shape (400,)
 y_pred_train : 
  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.

In [ ]:
#### With Pre Process

In [2]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
from keras.applications.xception import preprocess_input

from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

from keras.applications.xception import Xception

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_Xception_PP_data4.h5'

train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Xception network
    model = Xception(weights='imagenet', include_top=False)
    
    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_Xception_PP_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_Xception_PP_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_Xception_PP_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    preprocess_input(train_data)   

    validation_data = np.load(open('DR1_bottleneck_features_validation_Xception_PP_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    preprocess_input(validation_data)   

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_train shape',y_pred_validation.shape)
    print(' y_pred_train : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

Epoch 39/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 40/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 41/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 42/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 43/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 44/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 45/50
1000/1000 [==============================] - 32s 32ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 46/50
1000/1000 [==================

/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


validation loss: 8.059047746974617
validation accuracy: 0.5
 y_pred_train shape (400,)
 y_pred_train : 
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.